<a href="https://colab.research.google.com/github/Corassini/Curso_deep_learning/blob/main/Exercicio_selecao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Abrindo o Arquivo CISI

In [ ]:
# ! cd /content/sample_data

In [1]:
!tar xf cisi.tar.gz

In [2]:
!pwd

/content


In [3]:
!ls

CISI.ALL  CISI.BLN  CISI.QRY  CISI.REL	cisi.tar.gz  sample_data


### Lendo os arquivos CISI e separando em diferentes variáveis

In [40]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [38]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer() 

#Criando uma lista de documentos e de buscas
documents = []
queries = []

In [41]:
#Tokenizando os documentos, retirando stop words e apendando na lista de documentos
#Utilização do stemmer - reduzir às palavras a seu radical
with open('/content/CISI.ALL','r') as f:
    content = f.read().lower()
    tokens = [stemmer.stem(token) for token in word_tokenize(content) if token not in stop_words]
    documents.append(tokens)

In [59]:
with open('/content/CISI.QRY', 'r') as f:
    content_qry = f.read().lower()
    tokens_q = [stemmer.stem(token) for token in word_tokenize(content_qry) if token not in stop_words]
    queries.append(tokens_q)

In [47]:
# Mesmo código do que o de cima, mas o de cima está mais compacto
# 
# with open('/content/CISI.QRY', 'r') as f:
#     content = f.read().lower()
#     for query in content.split('\n'):
#         tokens_q = []
#         for token in word_tokenize(query):
#             if token not in stop_words:
#                 tokens_q.append(stemmer.stem(token))
#         queries.append(tokens_q)

### Implementando o BM25

In [21]:
from gensim.summarization.bm25 import BM25

In [48]:
bm25 = BM25(documents)

In [61]:
#Calculando o IDF
idf_values = bm25.idf.values()
idf_sum = 0
for value in idf_values:
    idf_sum += float(value)
idf_mean = idf_sum / len(idf_values)

In [62]:
#Calculando o score
scores = []
for query in queries:
  query_scores = bm25.get_scores(query,idf_mean)
  scores.append(query_scores)

In [63]:
for i, query in enumerate(queries):
    ranked_documents = sorted(range(len(scores[i])), key=lambda j: scores[i][j], reverse=True)
    print("Consulta: ", query[:5])
    print("Documentos mais relevantes: ", ranked_documents[:10])

Consulta:  ['.i', '1']
Documentos mais relevantes:  [0]
Consulta:  ['.w']
Documentos mais relevantes:  [0]
Consulta:  ['problem', 'concern', 'make', 'descript', 'titl']
Documentos mais relevantes:  [0]
Consulta:  ['difficulti', 'involv', 'automat', 'retriev', 'articl']
Documentos mais relevantes:  [0]
Consulta:  ['approxim', 'titl', '?']
Documentos mais relevantes:  [0]
Consulta:  ['usual', 'relev', 'content', 'articl', 'titl']
Documentos mais relevantes:  [0]
Consulta:  ['.i', '2']
Documentos mais relevantes:  [0]
Consulta:  ['.w']
Documentos mais relevantes:  [0]
Consulta:  ['actual', 'pertin', 'data', ',', 'oppos']
Documentos mais relevantes:  [0]
Consulta:  [',', 'retriev', 'automat', 'respons', 'inform']
Documentos mais relevantes:  [0]
Consulta:  ['.i', '3']
Documentos mais relevantes:  [0]
Consulta:  ['.w']
Documentos mais relevantes:  [0]
Consulta:  ['inform', 'scienc', '?', 'give', 'definit']
Documentos mais relevantes:  [0]
Consulta:  ['.i', '4']
Documentos mais relevantes:  